In [3]:
!pip install --upgrade --force-reinstall numpy==1.26.4 scipy==1.13.1 pandapower==2.14.6

  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached scipy-1.13.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
  Using cached pandapower-2.14.6.zip (13.2 MB)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 85.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 44.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 64.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 103.6 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.4/91.4 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
import pandapower as pp
import pandapower.networks as pn
import pandas as pd
import numpy as np
from tqdm import tqdm
import logging

logging.getLogger("pandapower").setLevel(logging.ERROR)

BASE_MVA = 100 

def generate_opf_dataset_pp(n_samples=200, perturb_range=(0.8, 1.0), save_path="test_data_118.csv"):
    net = pn.case118() 

    nominal_p = net.load.p_mw.values.copy()
    nominal_q = net.load.q_mvar.values.copy()

    data = []

    with tqdm(total=n_samples, desc="Generating AC-OPF dataset", ncols=100) as pbar:
        while len(data) < n_samples:

            scale = np.random.uniform(*perturb_range, size=len(net.load))
            net.load.p_mw = nominal_p * scale
            net.load.q_mvar = nominal_q * scale

            try:
                pp.runopp(net, verbose=False)

                if net.OPF_converged:
                    P = net.res_bus.p_mw.values / BASE_MVA
                    Q = net.res_bus.q_mvar.values / BASE_MVA
                    V = net.res_bus.vm_pu.values
                    theta = net.res_bus.va_degree.values

                    sample = np.concatenate([P, Q, V, theta]).astype(np.float32)
                    data.append(sample)
                    pbar.update(1)

            except:
                continue

    n_buses = len(net.bus)
    columns = (
        [f"P{i}" for i in range(n_buses)] +
        [f"Q{i}" for i in range(n_buses)] +
        [f"V{i}" for i in range(n_buses)] +
        [f"Theta{i}" for i in range(n_buses)]
    )

    df = pd.DataFrame(data, columns=columns)
    df.to_csv(save_path, index=False)

    print(f"\nGenerated {len(df)} AC-OPF samples")
    print(f"Saved to '{save_path}'")

    return df
df = generate_opf_dataset_pp(n_samples=5000, perturb_range=(0.8, 1.0))

Generating AC-OPF dataset: 100%|██████████████████████████████| 5000/5000 [1:12:43<00:00,  1.15it/s]



Generated 5000 AC-OPF samples
Saved to 'test_data_118.csv'
